In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pickle
import clean_data
import random
import train
import json
from tqdm.notebook import tqdm
from model import ResNet28
import time
import torch

cols = ['BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE', 'ECCENTRICITY',
       'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'MEAN_MOTION', 'SUNSPOTS_1D',
       'SUNSPOTS_3D', 'SUNSPOTS_7D', 'AIR_MONTH_AVG_TEMP',
       'WATER_MONTH_AVG_TEMP', 'year', 'month_sin', 'month_cos', 'hour_sin',
       'hour_cos', 'minute_sin', 'minute_cos', 'second_sin', 'second_cos',
       'ms_sin', 'ms_cos', 'year_y', 'month_sin_y', 'month_cos_y',
       'hour_sin_y', 'hour_cos_y', 'minute_sin_y', 'minute_cos_y',
       'second_sin_y', 'second_cos_y', 'ms_sin_y', 'ms_cos_y',
       'epoch_day_diff', 'epoch_sec_diff', 'epoch_ms_diff']

ycol_in = 'INCLINATION'
ycol_ec = 'ECCENTRICITY'
ycol_mm = 'MEAN_MOTION'
ycol_ra = 'RA_OF_ASC_NODE'
ycol_ap = 'ARG_OF_PERICENTER'
ycol_ma = 'MEAN_ANOMALY'

X_train = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/cleaned/x_train_sample_1000.pkl')
y_train = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/cleaned/y_train_sample_1000.pkl')
X_test = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/cleaned/x_test_sample_100.pkl')
y_test = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/cleaned/y_test_sample_100.pkl')

In [2]:
# cols_to_center = ['BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE', 'ECCENTRICITY',
#                   'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'MEAN_MOTION', 'SUNSPOTS_1D',
#                   'SUNSPOTS_3D', 'SUNSPOTS_7D', 'AIR_MONTH_AVG_TEMP',
#                   'WATER_MONTH_AVG_TEMP', 'year']
# X_train[cols_to_center] = X_train[cols_to_center] - 0.5
# X_test[cols_to_center] = X_test[cols_to_center] - 0.5

In [7]:
model_id = 'sample_n5_7in'
y_col = ycol_in

print(f'Training: {model_id}...')

config = {
    'model_identifier' : model_id,
    'model_path' : './data',
    'device' : 'cuda',
    'random_seed' : 0,
    'max_epochs' : 20,
    'do_validate' : True,
    'model_width' : 128,
    'optimizer' : {
        'lr' : 0.001,
        'weight_decay' : 0.00025,
        'momentum' : 0.9,
    },
    'scheduler' : {
        'max_lr' : 0.1,
        'steps_per_epoch' : 1846 , # Number of iterations within an epoch
        'div_factor' : 100, # initial_lr = max_lr / div_factor. default is 25
        'final_div_factor': 1e6, # min_lr = initial_lr/final_div_factor. default is 1e4
    },
    'train_params' : {
        'batch_size': 2000,
        'shuffle': True,
        'num_workers': 5,
        'pin_memory': True,
    },
    'test_params' : {
        'batch_size': 20000,
        'num_workers': 5,
        'pin_memory': True,
    },
}


model_id=config['model_identifier']
with open(f'data/{model_id}.cfg', 'w') as f:
    json.dump(config, f)

#Override model
model=ResNet28(input_size=len(cols), output_size=1, width=config['model_width'])
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=0.01,
                              weight_decay=0.00025)

model, mean_losses = train.train_model(X_train[cols], y_train[[y_col]],
                                   X_test[cols], y_test[[y_col]],
                                   config, force_train=True,
                                   net_override=model,
                                   optimizer_override=optimizer,)

Training: sample_n5_7in...
New model created
ResNet28(
  (stack1): ResNetStack(
    (stack): Sequential(
      (dense_1): ResnetDenseBlock(
        (dense1): Linear(in_features=37, out_features=128, bias=True)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (dense2): Linear(in_features=128, out_features=128, bias=True)
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dense3): Linear(in_features=128, out_features=128, bias=True)
        (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (dense_sc): Linear(in_features=37, out_features=128, bias=True)
          (bn_sc): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity_1a): ResnetIdentityBlock(
        (dense1): Linear(in_features=128, out_features=128,

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1846 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1846 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1846 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1846 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
# Second Cycle:
model_id = 'sample_n5_7in'
y_col = ycol_in

with open(f'data/{model_id}.cfg') as f:
    config = json.load(f)

additional_epochs = 10
config.update({'max_epochs': config['max_epochs'] + additional_epochs})
with open(f'data/{model_id}.cfg', 'w') as f:
    json.dump(config, f)
    
# Continue the same model... decreased lr
model, _ = train.load_model_only(config)
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=1e-8,
                              weight_decay=0.00025)

# Continue the same model...different optimizer
# model, _ = train.load_model_only(config)
# optimizer = torch.optim.SGD(model.parameters(), lr=6e-2, momentum=0)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1,
#                                                 epochs=additional_epochs,
#                                                 steps_per_epoch=config['scheduler']['steps_per_epoch'],
#                                                 div_factor=10,
#                                                 final_div_factor=100)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=config['scheduler']['max_lr'],
#                                                 epochs=additional_epochs,
#                                                 steps_per_epoch=config['scheduler']['steps_per_epoch'],
#                                                 div_factor=config['scheduler']['div_factor'],
#                                                 final_div_factor=config['scheduler']['final_div_factor'])

model, mean_losses = train.train_model(X_train[cols], y_train[[y_col]],
                                       X_test[cols], y_test[[y_col]],
                                       config, force_train=False,
                                       net_override=model,
                                       optimizer_override=optimizer,
                                       #scheduler_override=scheduler,
                                      )

Loading existing model
Loading existing model
ResNet28(
  (stack1): ResNetStack(
    (stack): Sequential(
      (dense_1): ResnetDenseBlock(
        (dense1): Linear(in_features=37, out_features=128, bias=True)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (dense2): Linear(in_features=128, out_features=128, bias=True)
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dense3): Linear(in_features=128, out_features=128, bias=True)
        (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (dense_sc): Linear(in_features=37, out_features=128, bias=True)
          (bn_sc): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity_1a): ResnetIdentityBlock(
        (dense1): Linear(in_features=128, out_features=128

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1846 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1846 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1846 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1846 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
   
# cols = ['BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE', 'ECCENTRICITY',
#        'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'MEAN_MOTION', 'SUNSPOTS_1D',
#        'SUNSPOTS_3D', 'SUNSPOTS_7D', 'AIR_MONTH_AVG_TEMP',
#        'WATER_MONTH_AVG_TEMP', 'year', 'month_sin', 'month_cos', 'hour_sin',
#        'hour_cos', 'minute_sin', 'minute_cos', 'second_sin', 'second_cos',
#        'ms_sin', 'ms_cos', 'year_y', 'month_sin_y', 'month_cos_y',
#        'hour_sin_y', 'hour_cos_y', 'minute_sin_y', 'minute_cos_y',
#        'second_sin_y', 'second_cos_y', 'ms_sin_y', 'ms_cos_y',
#        'epoch_day_diff', 'epoch_sec_diff', 'epoch_ms_diff']

# y_cols = {'sample_n5_1ra' : 'RA_OF_ASC_NODE',
#           'sample_n5_1ap' : 'ARG_OF_PERICENTER',
#           'sample_n5_1in' : 'INCLINATION',
#           'sample_n5_1mm' : 'MEAN_MOTION',
#           'sample_n5_1ec' : 'ECCENTRICITY',
#           'sample_n5_1ma' : 'MEAN_ANOMALY',
#          }

# for model_id, y_col in y_cols.items():
#     print(f'Training: {model_id}...')
    
#     config = {
#         'model_identifier' : model_id,
#         'model_path' : './data',
#         'device' : 'cuda',
#         'random_seed' : 0,
#         'max_epochs' : 100,
#         'do_validate' : True,
#         'model_width' : 256,
#         'optimizer' : {
#             'lr' : 0.01,
#             'weight_decay' : 0.00025,
#             'momentum' : 0.9,
#         },
#         'scheduler' : {
#             'max_lr' : 0.1,
#             'steps_per_epoch' : 1808, # Number of iterations within an epoch
#             'div_factor' : 100, # initial_lr = max_lr / div_factor. default is 25
#             'final_div_factor': 1e4, # min_lr = initial_lr/final_div_factor. default is 1e4
#         },
#         'train_params' : {
#             'batch_size': 2000,
#             'shuffle': True,
#             'num_workers': 5,
#             'pin_memory': True,
#         },
#         'test_params' : {
#             'batch_size': 20000,
#             'num_workers': 5,
#             'pin_memory': True,
#         },
#     }


#     model_id=config['model_identifier']
#     with open(f'data/{model_id}.cfg', 'w') as f:
#         json.dump(config, f)
        
#     model, mean_losses = train.train_model(X_train[cols], y_train[[y_col]],
#                                        X_test[cols], y_test[[y_col]],
#                                        config, force_train=False)

In [ ]:
# model_id='full_local_10c'

# with open(f'data/{model_id}.cfg') as f:
#     configurations = json.load(f)
# configurations

In [ ]:
# # model_id='full_local_10c'
# # c = {'lr' : 0.0001, 'model_identifier': model_id}
# # configurations.update(c)

# # model_id=configurations['model_identifier']
# # with open(f'data/{model_id}.cfg', 'w') as f:
# #     json.dump(configurations, f)
    
# cols = ['BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE', 'ECCENTRICITY',
#        'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'MEAN_MOTION', 'SUNSPOTS_1D',
#        'SUNSPOTS_3D', 'SUNSPOTS_7D', 'AIR_MONTH_AVG_TEMP',
#        'WATER_MONTH_AVG_TEMP', 'year', 'month_sin', 'month_cos', 'hour_sin',
#        'hour_cos', 'minute_sin', 'minute_cos', 'second_sin', 'second_cos',
#        'ms_sin', 'ms_cos', 'year_y', 'month_sin_y', 'month_cos_y',
#        'hour_sin_y', 'hour_cos_y', 'minute_sin_y', 'minute_cos_y',
#        'second_sin_y', 'second_cos_y', 'ms_sin_y', 'ms_cos_y',
#        'epoch_day_diff', 'epoch_sec_diff', 'epoch_ms_diff']

# ycol_ra = ['RA_OF_ASC_NODE']
# ycol_ap = ['ARG_OF_PERICENTER']
# ycol_in = ['INCLINATION']
# ycol_mm = ['MEAN_MOTION']
# ycol_ec = ['ECCENTRICITY']
# ycol_ma = ['MEAN_ANOMALY']

# model, mean_losses = train.train_model(X_train[cols], y_train[ycol_ra],
#                                        X_test[cols], y_test[ycol_ra],
#                                        configurations, force_train=True)